We start by activating the IPython extension and importing and aliasing a few useful components. We also select the monochromatic mode.

In [ ]:
%load_ext eradiate

import matplotlib.pyplot as plt
import numpy as np

import eradiate
from eradiate import unit_registry as ureg
from eradiate import scenes as ertsc
eradiate.set_mode("mono")

We set the computational mode to monochromatic, since we will be simulating scenes without atmospheres. Hence the correlated-*k* distribution method will not be useful here.

Next we set up convenience functions for plotting BRF results and visualizing camera renders.

In [ ]:
def show_camera(
    exp,
    measure_id,
    robust=True,
    add_colorbar=False,
    vmin=None,
    vmax=None,
):
    """
    Display the output of a monochromatic camera measure.
    """
    _, ax = plt.subplots(1, 1)
    exp.results[measure_id]["radiance"].squeeze(drop=True).plot.imshow(
        ax=ax,
        origin="upper",
        cmap="Greys_r",
        vmin=vmin,
        vmax=vmax,
        robust=robust,
        add_colorbar=add_colorbar,
    )
    ax.set_aspect(1)  # Ensures that pixels are represented as squares
    plt.show()
    plt.close()

def show_brf(exp, measure_id):
    """
    Display the BRF output of a distant radiance measure.
    """
    _, ax = plt.subplots(1, 1)
    exp.results[measure_id]["brf"].squeeze(drop=True).plot(
        ax=ax, x="vza", linestyle=":", marker="."
    )
    plt.show()
    plt.close()

In [ ]:
lambertian_surface=ertsc.surface.BasicSurface(
    shape=ertsc.shapes.RectangleShape(edges=10.0 * ureg.m),
    bsdf=ertsc.bsdfs.LambertianBSDF(reflectance=0.5),
)

In [ ]:
camera_oblique = ertsc.measure.PerspectiveCameraMeasure(
    id="camera_oblique",
    origin=[15, 15, 15] * ureg.m,
    target=[0, 0, 0] * ureg.m,
    up=[0, 0, 1],
    film_resolution=(320, 240),
    spp=512,
)

In [ ]:
exp = eradiate.experiments.CanopyExperiment(
    surface=lambertian_surface,
    measures=camera_oblique,
)

We run the simulation and use the convenience function defined above to visualize the result.

In [ ]:
eradiate.run(exp)
show_camera(exp, "camera_oblique")

In [ ]:
homogeneous_canopy = ertsc.biosphere.DiscreteCanopy.homogeneous(
    l_vertical=1.0 * ureg.m,
    l_horizontal=10.0 * ureg.m,
    lai=2.0,
    leaf_radius=10 * ureg.cm,
)

We create a new experiment object, which contains the canopy we just defined.

**Note:** We can now define the surface through its BSDF only, because the size of the rectangular shape is defined by the width of the canopy.

In [ ]:
exp = eradiate.experiments.CanopyExperiment(
    surface=ertsc.bsdfs.LambertianBSDF(reflectance=0.5),
    canopy=homogeneous_canopy,
    measures=camera_oblique,
)

We run the experiment and display the result.

In [ ]:
eradiate.run(exp)
show_camera(exp, "camera_oblique")

In [ ]:
hdistant = eradiate.scenes.measure.HemisphericalDistantMeasure(spp=10000)

In [ ]:
exp = eradiate.experiments.CanopyExperiment(
    surface=ertsc.bsdfs.LambertianBSDF(reflectance=0.5),
    illumination=ertsc.illumination.DirectionalIllumination(
        zenith=30.0 * ureg.deg,
        azimuth=45.0 * ureg.deg,
    ),
    canopy=homogeneous_canopy,
    measures=hdistant,
)
ds = eradiate.run(exp)
ds

We can now visualise the data quickly using xarray's built-in plotting facilities:

In [ ]:
ds["brf"].squeeze().plot()

In [ ]:
mdistant = ertsc.measure.MultiDistantMeasure.from_viewing_angles(
    id="toa_brf",
    zeniths=np.arange(-75, 76, 5),
    azimuths=45 * ureg.deg,
    spectral_cfg={"wavelengths": [550]},
    spp=10000,
)

exp = eradiate.experiments.CanopyExperiment(
    surface=ertsc.bsdfs.LambertianBSDF(reflectance=0.5),
    illumination=ertsc.illumination.DirectionalIllumination(
        zenith=30.0 * ureg.deg,
        azimuth=45.0 * ureg.deg,
    ),
    canopy=homogeneous_canopy,
    measures=mdistant,
)

ds = eradiate.run(exp)
ds

Visualisation is also greatly facilitated by xarray's plotting features. We explicitly use the `vza` (viewing zenith angle) coordinate as the *x* coordinate. We see the retro-reflective "hot spot" in the illumination direction (30°). Also note how variance typical of Monte Carlo methods appears: it can be reduced by increasing the sample count of the measure (`spp` parameter).

In [ ]:
ds.brf.plot(x="vza");

In [ ]:
exp = eradiate.experiments.CanopyExperiment(
    surface=ertsc.bsdfs.LambertianBSDF(reflectance=0.5),
    canopy=homogeneous_canopy,
    padding=1,
    measures=camera_oblique,
)

Let's run this experiment and visualise the results:

In [ ]:
eradiate.run(exp)
show_camera(exp, "camera_oblique")

Our unit cell is now surrounded by a row of clones of itself: this amounts to 8 clones. Let's increase padding to 2 (we now have 8 + 16 = 24 clones):

In [ ]:
exp = eradiate.experiments.CanopyExperiment(
    surface=ertsc.bsdfs.LambertianBSDF(reflectance=0.5),
    canopy=homogeneous_canopy,
    padding=2,
    measures=camera_oblique,
)
eradiate.run(exp)
show_camera(exp, "camera_oblique", add_colorbar=True)

We can see that the rendering time increases with padding. This is due to more pixels of the final image requiring the simulation of multiple scattering and rendering time should become approximately constant with larger padding values:

In [ ]:
exp = eradiate.experiments.CanopyExperiment(
    surface=ertsc.bsdfs.LambertianBSDF(reflectance=0.5),
    canopy=homogeneous_canopy,
    padding=25,
    measures=camera_oblique,
)thethe
eradiate.run(exp)
show_camera(exp, "camera_oblique", vmin=0.0, add_colorbar=True)